## Testing MLOps engineer permissions

### Positive test cases:
1. Should be able to deploy a model to SageMake endpoint.
2. Should be able to setup a model monitor job.
3. Should be able to create, read, update and delete model package from model registry.
4. Should be able to deploy a sagemaker project.

### Negative test cases:
1. Should NOT be able to spin up a training job.
2. Should NOT be able to spin up a processing job.


## Let us execute the positive test cases:

### Positive case 1: Should be able to deploy a model to SageMake endpoint.

In [ ]:
import os
import boto3
import re
import json
import sagemaker
from sagemaker import get_execution_role, session
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.image_uris import retrieve
from sagemaker.model_monitor import DataCaptureConfig

sm_session = sagemaker.Session()
region = sm_session.boto_region_name

role = get_execution_role()
print("Role ARN: {}".format(role))

bucket = sm_session.default_bucket()
prefix = "lab-4/test/model"

model_file = open("model/xgb-churn-prediction-model.tar.gz", "rb")
s3_key = os.path.join(prefix, "xgb-churn-prediction-model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(model_file)

data_capture_prefix = "{}/model-monitor/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)

endpoint_name = "xgb-churn-pred-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

In [ ]:
model_name = "xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = "https://{}.s3-{}.amazonaws.com/{}/xgb-churn-prediction-model.tar.gz".format(
    bucket, region, prefix
)

image_uri = retrieve("xgboost", region, "0.90-1")

model = Model(image_uri=image_uri, model_data=model_url, role=role)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

### Positive case 2: Should be able able to setup a model monitor job.

In [ ]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + "model-monitor/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = "s3://{}/{}".format(bucket, baseline_data_prefix)
baseline_results_uri = "s3://{}/{}".format(bucket, baseline_results_prefix)
print("Baseline data uri: {}".format(baseline_data_uri))
print("Baseline results uri: {}".format(baseline_results_uri))

training_data_file = open("model_monitor/test_data/training-dataset-with-header.csv", "rb")
s3_key = os.path.join(baseline_prefix, "data", "training-dataset-with-header.csv")
boto3.Session().resource("s3").Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri + "/training-dataset-with-header.csv",
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True,
)

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator
print(predictor)

mon_schedule_name = "xgb-churn-pred-model-monitor-schedule-" + strftime(
    "%Y-%m-%d-%H-%M-%S", gmtime()
)
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

### Positive test case 3: Should be able to Register, read, update and delete model package from sagemaker model registry


#### 3.1 Register model in model registry

In [ ]:
from sagemaker.workflow.parameters import ParameterString

sm_client = boto3.client("sagemaker")
response = sm_client.create_model_package_group(
    ModelPackageGroupName='project-1',
    ModelPackageGroupDescription='abc-g'
)

print(response)

In [ ]:
response = sm_client.create_model_package(
    ModelPackageGroupName='arn:aws:sagemaker:us-east-1:154856345127:model-package-group/abc',
    InferenceSpecification={
        'Containers': [
            {
                'Image': image_uri,
                'ModelDataUrl': model_url,
                'Framework': 'XGBOOST',
                'FrameworkVersion': '1.0',
            },
        ],
        'SupportedTransformInstanceTypes': [
            'ml.m4.xlarge'
        ],
        'SupportedRealtimeInferenceInstanceTypes': [
            'ml.t2.medium'
        ],
        'SupportedContentTypes': [
            'text/csv',
        ],
        'SupportedResponseMIMETypes': [
            'text/csv',
        ]
    },
    ModelApprovalStatus='PendingManualApproval',
)

print(response)

#### 3.2 Delete a model package from model registry